In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import numpy as np
import warnings
warnings.simplefilter('ignore')

In [3]:
image_directory = None
depth_directory = None
video_path = None
json_path = None
csv_path = None
txt_path = None

weights_dir_path = '../weights'
optical_flow_checkpoint = os.path.join('/Vol0/user/f.konokhov/tfoptflow/tfoptflow/tmp',
                                       'pwcnet.ckpt-84000')

depth_estimator_name = 'struct2depth'
depth_checkpoint = os.path.join(weights_dir_path, 'model-199160')
#depth_estimator_name = 'senet'

computation_kwargs = dict(
    cuda_visible_devices=0,
)

sys.path.append(os.path.abspath(os.pardir))
sys.path.append(os.path.join(os.path.abspath(os.pardir), 'submodules/tf_models/research/struct2depth'))
sys.path.append(os.path.join(os.path.abspath(os.pardir), 'submodules/tfoptflow/tfoptflow'))

In [4]:
from prepare_dataset.dataset_builder import ImagesDatasetBuilder

### Initialize

In [6]:
sequence_directory = '03'

builder = ImagesDatasetBuilder(
     sequence_directory=sequence_directory,
     build_from='DISCOMAN',
     image_directory='/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/03/image_2',
     mode=ImagesDatasetBuilder.TEST,
     estimate_optical_flow=True,
     optical_flow_estimator_name='pwc',
     optical_flow_checkpoint=optical_flow_checkpoint,
     estimate_depth=False,
     depth_estimator_name=depth_estimator_name,
     depth_checkpoint=depth_checkpoint,
     memory_safe=True,
     **computation_kwargs)
builder._configure()

Using FPS rate = 30


### Build

In [7]:
builder.build() # "Do everything" big red button

Using FPS rate = 30
Estimate optical flow
Model:   <class 'prepare_dataset.estimator.PWCOpticalFlowEstimator'>
Weights: /Vol0/user/f.konokhov/tfoptflow/tfoptflow/tmp/pwcnet.ckpt-84000
Output:  03/optical_flow_stride1


Computation settings:



Using TensorFlow backend.


CUDA visible devices:     0
Available GPUs:           /job:localhost/replica:0/task:0/device:GPU:0
Allow growth:             True
GPU memory fraction:      0.33
Number of CPU:            8
Number of CPU threads:    16

Random seed:              42



Building model...
Instructions for updating:
Use keras.layers.conv2d instead.


W0604 16:19:07.163080 47974054530688 deprecation.py:323] From /Vol0/user/a.vorontsova/odometry/submodules/tfoptflow/tfoptflow/model_pwcnet.py:1173: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.conv2d instead.


Instructions for updating:
Colocations handled automatically by placer.


...

W0604 16:19:13.639808 47974054530688 deprecation.py:323] From /Vol0/user/a.vorontsova/miniconda3/lib/python3.6/site-packages/tensorflow/python/training/saver.py:1266: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


INFO:tensorflow:Restoring parameters from /Vol0/user/f.konokhov/tfoptflow/tfoptflow/tmp/pwcnet.ckpt-84000


I0604 16:19:13.743767 47974054530688 saver.py:1270] Restoring parameters from /Vol0/user/f.konokhov/tfoptflow/tfoptflow/tmp/pwcnet.ckpt-84000


... model loaded
Estimator(dir=03/optical_flow_stride1, image_manager=ImageManager(dir=/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/03/image_2, image_height=375, image_width=1242, stride=1, sample=True, step=1), checkpoint=/Vol0/user/f.konokhov/tfoptflow/tfoptflow/tmp/pwcnet.ckpt-84000)


Optical flow estimation: 100%|██████████| 800/800 [02:23<00:00,  4.01it/s]


In [8]:
builder.dataframe.to_csv(sequence_directory + '/df.csv')

### Test Visual Odometry

In [9]:
target_size = (96, 320)

In [10]:
from prepare_dataset.generator_factory import GeneratorFactory
dataset = GeneratorFactory(
    csv_name=('df.csv'),
    dataset_root='/Vol0/user/a.vorontsova/odometry/notebooks',
    train_sequences=[sequence_directory],
    val_sequences=[sequence_directory],
    target_size=target_size,
    x_col=['path_to_optical_flow'],
    y_col=['euler_x', 'euler_y', 'euler_z', 'x', 'y', 'z'],
    image_columns=['path_to_optical_flow'],
    load_modes=['flow_xy'],
    preprocess_modes=['flow_xy'],
    val_sampling_step=1,
    cached_imgs=None,
    batch_size=32
)

100%|██████████| 1/1 [00:00<00:00, 82.90it/s]

No cache


In [11]:
dataset.df_val.path_to_optical_flow = dataset.df_val.path_to_optical_flow.str.replace(
    '/{}/'.format(sequence_directory), '/')

In [12]:
from models.model_factory import ModelFactory

model_factory = ModelFactory(
    target_size,
    channels_counts=dataset.channels_counts,
    lr=0.001,
    loss='mae',
    scale_rotation=50
)
model = model_factory.construct_pretrained_model(
    '/Vol0/user/f.konokhov/FlavorNet/models/kitty_train0289/weights/models-kitty_train0289-ls_vo_pwc_classic.100-0.101066.hdf5')

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


W0604 16:24:14.079601 47974054530688 deprecation.py:323] From /Vol0/user/a.vorontsova/miniconda3/lib/python3.6/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Load weights from /Vol0/user/f.konokhov/FlavorNet/models/kitty_train0289/weights/models-kitty_train0289-ls_vo_pwc_classic.100-0.101066.hdf5
Instructions for updating:
Use tf.cast instead.


W0604 16:24:40.796696 47974054530688 deprecation.py:323] From /Vol0/user/a.vorontsova/miniconda3/lib/python3.6/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


In [13]:
generator = dataset.get_val_generator()
model_output = model.predict_generator(generator, steps=len(generator))

No cache


In [14]:
from linalg.trajectory import RelativeTrajectory

In [15]:
keys = ['euler_x', 'euler_y', 'euler_z', 'x', 'y', 'z']

In [16]:
gt = builder.dataframe[keys].to_dict(orient='records')

In [17]:
gt_relative_trajectory = RelativeTrajectory()
gt_relative_trajectory.from_euler_angles(gt)
gt_global_trajectory = gt_relative_trajectory.relative_to_global()

In [18]:
predictions = [dict(zip(keys, values)) for values in np.concatenate(model_output, 1)]

In [19]:
predicted_relative_trajectory = RelativeTrajectory()
predicted_relative_trajectory.from_euler_angles(predictions)
predicted_global_trajectory = predicted_relative_trajectory.relative_to_global()
predicted_global_trajectory.plot('tmp.html')

In [20]:
from evaluation.visualization_utils import visualize_trajectory
visualize_trajectory(predicted_global_trajectory, title=sequence_directory, is_3d=True, file_name='tmp_3d.html')
visualize_trajectory(predicted_global_trajectory, title=sequence_directory, is_3d=False, file_name='tmp_2d.html')

In [26]:
from evaluation.visualization_utils import visualize_trajectory
visualize_trajectory(gt_global_trajectory, title=sequence_directory, is_3d=True, file_name='gt_3d.html')
visualize_trajectory(gt_global_trajectory, title=sequence_directory, is_3d=False, file_name='gt_2d.html')

In [31]:
from evaluation.visualization_utils import visualize_trajectory_with_gt
visualize_trajectory_with_gt(gt_global_trajectory, predicted_global_trajectory, title=sequence_directory, is_3d=True, file_name='tmp_gt_3d.html')
visualize_trajectory_with_gt(gt_global_trajectory, predicted_global_trajectory, title=sequence_directory, is_3d=False, file_name='tmp_gt_2d.html')

In [24]:
from evaluation.metrics import ATE
ATE(gt_global_trajectory, predicted_global_trajectory)

444.5658312587294